In [10]:
import numpy as np
import sys
from astropy.io import fits
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import glob
import gzip
import os
from scipy.interpolate import interp1d
import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import fftconvolve
from PyAstronomy.pyasl import rotBroad
import numpy as np
from math import sin, pi
from scipy.special import erf                               # Error function
from scipy.signal import fftconvolve 
from lmfit import Parameters, minimize


In [11]:
def read_HERMES(infile):
    #print("%s: Input file is a HERMES file." % infile)
    header = fits.getheader(infile)

    #bjd = header['MJD-OBS']
    # for files with standard wavelegth array
    if ((header['CTYPE1'] == 'WAVELENGTH') or (header['CTYPE1'] == 'AWAV')):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = crval + np.arange(0, naxis1) * cdelt

    # for files that are given in logarithmic wl array
    if (header['CTYPE1'] == 'log(wavelength)'):
        flux = fits.getdata(infile, byteorder='little')
        crval = header['CRVAL1']
        cdelt = header['CDELT1']
        naxis1 = header['NAXIS1']
        wave = np.exp(crval + np.arange(0, naxis1)*cdelt)
    else:
        print("Could not read in HERMES fits file - unknown file type.")
        sys.exit()
    return wave, flux

#reading from linelist to fit
def read_line_list(filename):
    line_centers = []
    line_widths = []

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  #removing whitespaces
            if not line:
                continue  #skipping empty lines
            parts = line.split()
            center = float(parts[0])
            if len(parts) > 1:
                width = float(parts[1])
            else:
                width = 10.0  #default
            line_centers.append(center)
            line_widths.append(width)

    return line_centers, line_widths

In [12]:
def gauss(x,a,center,R, gamma):
  sigma = sigma = 4471/ (2.0 * R * np.sqrt(2.0 * np.log(2))) 
  return a*np.exp(-(x-center)**2/(2*sigma**2)) + gamma

def generate_data(wave, flux, line_centers, line_widths, wavelength_slices):
    interp_func = interp1d(wave, flux, kind='linear')
    wave_slices = []
    flux_slices = []
    for center, width in zip(line_centers, line_widths):
        new_wave = np.linspace(center - width, center + width, wavelength_slices)
        new_flux = interp_func(new_wave)
        wave_slices.append(new_wave)
        flux_slices.append(new_flux)
    return np.concatenate(wave_slices), np.concatenate(flux_slices)

In [13]:
import warnings
class Model_broad:
    def __init__(self, wave, flux):
        self.x = wave
        self.y = flux


def Broaden(model, vsini, epsilon=0.5, linear=False, findcont=False):
    # Remove NaN values from the flux array and corresponding wavelength values
    non_nan_idx = ~np.isnan(model.y)
    wvl = model.x[non_nan_idx]
    flx = model.y[non_nan_idx]
    
    dwl = wvl[1] - wvl[0]
    binnu = int(np.floor((((vsini/10)/ 299792.458) * max(wvl)) / dwl)) + 1 #adding extra bins for error handling
    #validIndices = np.arange(len(flx)) + binnu => this was used in rotbroad as a user cond ==> this is always on here
    front_fl = np.ones(binnu) * flx[0]
    end_fl = np.ones(binnu) * flx[-1]
    flux = np.concatenate((front_fl, flx, end_fl))

    front_wv = (wvl[0] - (np.arange(binnu) + 1) * dwl)[::-1]
    end_wv = wvl[-1] + (np.arange(binnu) + 1) * dwl
    wave = np.concatenate((front_wv, wvl, end_wv))

    if not linear:
        x = np.logspace(np.log10(wave[0]), np.log10(wave[-1]), len(wave))
    else:
        x = wave
        
    if findcont:
        # Find the continuum
        model.cont = np.ones_like(flux)  # Placeholder for continuum finding
        
    # Make the broadening kernel
    dx = np.log(x[1] / x[0])
    c = 299792458  # Speed of light in m/s
    lim = vsini / c
    if lim < dx:
        warnings.warn("vsini too small ({}). Not broadening!".format(vsini))
        return Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    
    d_logx = np.arange(0.0, lim, dx)
    d_logx = np.concatenate((-d_logx[::-1][:-1], d_logx))
    alpha = 1.0 - (d_logx / lim) ** 2
    B = (1.0 - epsilon) * np.sqrt(alpha) + epsilon * np.pi * alpha / 4.0  # Broadening kernel
    B /= np.sum(B)  # Normalize

    # Do the convolution
    broadened = Model_broad(wave.copy(), flux.copy())  # Create a copy of the Model object
    broadened.y = fftconvolve(flux, B, mode='same')
    
    return broadened

def macro_broaden(xdata, ydata, vmacro):
    c = 299792458 #~constants.c.cgs.value * units.cm.to(units.km)
    sq_pi = np.sqrt(np.pi)
    lambda0 = np.median(xdata)
    xspacing = xdata[1] - xdata[0]
    mr = vmacro * lambda0 / c
    ccr = 2 / (sq_pi * mr)

    px = np.arange(-len(xdata) / 2, len(xdata) / 2 + 1) * xspacing
    pxmr = abs(px) / mr
    profile = ccr * (np.exp(-pxmr ** 2) + sq_pi * pxmr * (erf(pxmr) - 1.0))

    before = ydata[int(-profile.size / 2 + 1):]
    after = ydata[:int(profile.size / 2 +1)] #add one to fix size mismatch
    extended = np.r_[before, ydata, after]

    first = xdata[0] - float(int(profile.size / 2.0 + 0.5)) * xspacing
    last = xdata[-1] + float(int(profile.size / 2.0 + 0.5)) * xspacing
    
    x2 = np.linspace(first, last, extended.size)  #newdata x array ==> handles edge effects

    conv_mode = "valid"

    newydata = fftconvolve(extended, profile / profile.sum(), mode=conv_mode)

    return newydata


def generate_broaden(params, line_centers, line_widths, wavelength_slices):
    model_slices = []
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        wave = np.linspace(center - width, center + width, wavelength_slices)
        
        instrum = gauss(wave, params[f'a{i}'], params[f'center{i}'], 20000, params[f'gamma{i}']) #resolution is still hardcoded R=20000 change accordingly
        broad_rot = Broaden(Model_broad(wave, instrum), params['vsini'])
        
        broad_macro = macro_broaden(broad_rot.x, broad_rot.y, params[f'vmacro{i}']) #macro broad restores the same wave array as input  
        
        interp = interp1d(broad_rot.x, broad_macro, kind= 'linear')
        broad_flux = interp(wave)
        model_slices.append(broad_flux)
        
    return  np.concatenate(model_slices)



In [14]:
def objective(params, wave, flux, line_centers, line_widths, wavelength_slices):
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    model = generate_broaden(params, line_centers, line_widths, wavelength_slices)
    return flux_data - model

def fit_lines(wave, flux, line_centers, line_widths, wavelength_slices):
    params = Parameters()
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    for i, (center, width) in enumerate(zip(line_centers, line_widths)):
        params.add(f'a{i}', value=-1)   # Initial guess for amplitude
        params.add(f'center{i}', value=center)  # Initial guess for center
        params.add(f'gamma{i}', value=1)
        params.add(f'vmacro{i}', value=150000, min = 0, max = 500000)
    params.add('vsini', value=150000, min = 0, max = 500000)
    

    result = minimize(objective, params=params, args=(wave_data, flux_data, line_centers, line_widths, wavelength_slices))
    return result

In [15]:
wave, flux = read_HERMES('00943975_HRF_OBJ_ext_CosmicsRemoved_log_merged_cf_norm.fits')
line_centers, line_widths = read_line_list('line_list.txt')
result = fit_lines(wave, flux, line_centers, line_widths, wavelength_slices=1000)
result

### errors with emcee

In [ ]:
import emcee
import corner
from tqdm import tqdm

# Define the log likelihood function
def log_likelihood(params, wave, flux, line_centers, line_widths, wavelength_slices):
    wave_data, flux_data = generate_data(wave, flux, line_centers, line_widths, wavelength_slices)
    model = generate_broaden(params, line_centers, line_widths, wavelength_slices)
    residuals = flux_data - model
    # Assuming Gaussian uncertainties, calculate the log likelihood
    sigma = np.sqrt(np.mean(residuals**2))  # Assuming constant uncertainty for simplicity
    log_likelihood = -0.5 * np.sum((residuals / sigma)**2)
    return log_likelihood

# Define the log prior function
def log_prior(params):
    # Assuming flat priors for simplicity
    # Check if parameters are within allowed ranges
    for i, value in enumerate(params):
        key = list(result.params.keys())[i]
        if key.startswith('vmacro'):
            if value < 0 or value > 500000:
                return -np.inf
        elif key == 'vsini':
            if value < 0 or value > 500000:
                return -np.inf
    return 0.0

# Define the log probability function
def log_probability(params, wave, flux, line_centers, line_widths, wavelength_slices):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, wave, flux, line_centers, line_widths, wavelength_slices)

# Set up the initial positions of the walkers
nwalkers = 32  # Number of walkers
ndim = len(result.params)  # Dimensionality of the parameter space
initial_params = [np.array(list(result.params.valuesdict().values())) + 1e-4 * np.random.randn(ndim) for _ in range(nwalkers)]

# Set up the emcee sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(wave, flux, line_centers, line_widths, 1000))

# Run the MCMC sampler with progress indicators
nsteps = 500  # Number of steps
with tqdm(total=nsteps) as pbar:
    for _ in sampler.sample(initial_params, iterations=nsteps):
        pbar.update(1)

# Extract the chain and discard burn-in
chain = sampler.get_chain()
burnin = 100  # Number of burn-in steps
samples = sampler.get_chain(discard=burnin, flat=True)

# Plot the results using the corner plot
labels = [f"{param}" for param in result.params.keys()]
corner.corner(samples, labels=labels, truths=list(result.params.valuesdict().values()))